## 폴더 설정, 파일 설정

In [3]:
import tkinter as tk
from tkinter import ttk, filedialog
from PIL import Image, ImageTk
from ultralytics import YOLO
import cv2
import os

# YOLO 모델 로드
model = YOLO('C:/KDT_SF6/vision_ai/best (1).pt')  # 모델 경로를 여기에 설정하세요

# 사용자 정의 클래스 이름
custom_names = {0: 'yaozhe', 1: 'waist folding', 2: 'punching hole', 3: 'welding line', 
                4: 'crescent gap', 5: 'water spot', 6: 'oil spot', 7: 'silk spot', 
                8: 'inclusion', 9: 'rolled pit', 10: 'crease', 11: 'd'}

# 클래스별 색상 설정
color_dict = {
    0: (255, 0, 0),   # yaozhe - 빨강
    1: (0, 255, 0),   # waist folding - 초록
    2: (0, 0, 255),   # punching hole - 파랑
    # 나머지 클래스 색상 추가
}

# 이미지 저장 경로 설정
save_path = 'C:/KDT_SF6/vision_ai/바운딩 박스 텍스트'  # 저장할 폴더 경로

# 총합을 저장할 딕셔너리 (초기값은 0)
total_sums = {key: 0 for key in custom_names.values()}

# 탐지 결과를 저장할 딕셔너리 (초기값 0)
detection_summary = {key: 0 for key in custom_names.values()}  # 각 클래스별 탐지 개수 초기화

# GUI 초기 설정
window = tk.Tk()
window.title("YOLO 이미지 탐지")
window.geometry("1200x800")

# 이미지 표시를 위한 라벨 추가 (좌측에 이미지 배치)
image_label = tk.Label(window)
image_label.grid(row=0, column=0, padx=10, pady=10, sticky="nw")

# 결과 레이블 (사진의 우측에 배치)
result_label = tk.Label(window, text="")
result_label.grid(row=0, column=1, padx=10, pady=10, sticky="nw")  # 이미지의 바로 우측에 배치

# Treeview 생성 및 열 설정 (하단에 위치)
tree_frame = tk.Frame(window)
tree_frame.grid(row=2, column=0, columnspan=2, sticky="nsew", padx=10, pady=10)

# 스크롤바 생성
tree_scroll = tk.Scrollbar(tree_frame)
tree_scroll.pack(side=tk.RIGHT, fill=tk.Y)

tree = ttk.Treeview(tree_frame, columns=('name', 'waist_folding', 'punching_hole', 
                                         'welding_line', 'crescent_gap', 'water_spot', 
                                         'oil_spot', 'silk_spot', 'inclusion', 
                                         'rolled_pit', 'crease'), show='headings',
                    yscrollcommand=tree_scroll.set)

# 스크롤바에 트리뷰 연결
tree_scroll.config(command=tree.yview)

# Treeview 열 제목 설정
tree.heading("name", text='name')
tree.heading("waist_folding", text="waist_folding")
tree.heading("punching_hole", text="punching_hole")
tree.heading("welding_line", text="welding_line")
tree.heading("crescent_gap", text="crescent_gap")
tree.heading("water_spot", text="water_spot")
tree.heading("oil_spot", text="oil_spot")
tree.heading("silk_spot", text="silk_spot")
tree.heading("inclusion", text="inclusion")
tree.heading("rolled_pit", text="rolled_pit")
tree.heading("crease", text="crease")

# Treeview 열 너비 설정
tree.column("name", width=120)
tree.column("waist_folding", width=100)
tree.column("punching_hole", width=100)
tree.column("welding_line", width=100)
tree.column("crescent_gap", width=100)
tree.column("water_spot", width=100)
tree.column("oil_spot", width=100)
tree.column("silk_spot", width=100)
tree.column("inclusion", width=100)
tree.column("rolled_pit", width=100)
tree.column("crease", width=100)

# Treeview 배치
tree.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)

# 총합 업데이트 함수
def update_summary():
    summary_tree.delete(*summary_tree.get_children())  # 기존 항목 삭제
    new_data = (
        "Total",
        total_sums['waist folding'],
        total_sums['punching hole'],
        total_sums['welding line'],
        total_sums['crescent gap'],
        total_sums['water spot'],
        total_sums['oil spot'],
        total_sums['silk spot'],
        total_sums['inclusion'],
        total_sums['rolled pit'],
        total_sums['crease']
    )
    summary_tree.insert("", tk.END, values=new_data)  # 총합 업데이트

# 파일을 순차적으로 확인하여 예측을 수행하는 함수
def check_for_new_image():
    global file_list, file_index
    if file_list and file_index < len(file_list):
        file_path = os.path.join(selected_folder, file_list[file_index])
        predict_image(file_path)
        file_index += 1

    # 더 이상 파일이 없으면 종료
    if file_index >= len(file_list):
        result_label.config(text="모든 파일을 처리하였습니다.")
        return

    # 2초마다 이 함수를 다시 호출
    window.after(2000, check_for_new_image)

# 폴더 선택 및 이미지 처리 함수
def select_folder():
    global file_list, file_index, selected_folder
    selected_folder = filedialog.askdirectory()  # 폴더 선택 다이얼로그
    if selected_folder:
        file_list = [f for f in os.listdir(selected_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]
        file_index = 0
        if file_list:
            check_for_new_image()

# 개별 파일 선택 및 예측 수행 함수
def upload_and_predict():
    file_path = filedialog.askopenfilename(title="이미지 파일 선택", filetypes=[("Image Files", "*.png;*.jpg;*.jpeg")])
    if file_path:
        predict_image(file_path)

def predict_image(file_path):
    try:
        # 이미지 로드 및 YOLO 예측 수행
        results = model.predict(file_path, save=False, imgsz=640, conf=0.2)

        image = cv2.imread(file_path)  # 이미지 로드
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # OpenCV 이미지를 RGB로 변환
        h, w, c = image.shape  # 이미지 크기 저장 (높이, 너비, 채널)

        final_boxes = []  # 최종 박스를 저장할 리스트

        # 바운딩 박스 처리
        for r in results:
            r.names = custom_names

            boxes = r.boxes.xyxy  # 바운딩 박스 좌표
            cls = r.boxes.cls  # 클래스 번호
            conf = r.boxes.conf  # 신뢰도
            cls_dict = r.names  # 클래스 이름

            # 바운딩 박스 및 클래스 이름 처리
            for box, cls_number, conf_value in zip(boxes, cls, conf):
                x1, y1, x2, y2 = box  # 바운딩 박스 좌표
                x1_int, y1_int = int(x1.item()), int(y1.item())  # 좌상단 좌표
                x2_int, y2_int = int(x2.item()), int(y2.item())  # 우하단 좌표
                cls_name = cls_dict[int(cls_number.item())]  # 클래스 이름
                conf_number = float(conf_value.item())  # 신뢰도

                # 바운딩 박스를 최종 리스트에 추가
                final_boxes.append({"coords": (x1_int, y1_int, x2_int, y2_int), 
                                    "cls_name": cls_name, "conf": conf_number})

        # 이미지 중심 좌표 계산
        center_image_x = w // 2
        center_image_y = h // 2

        # 최종적으로 남은 박스만 그리기
        for box in final_boxes:
            x1_int, y1_int, x2_int, y2_int = box["coords"]
            cls_name = box["cls_name"]
            conf_number = box["conf"]

            label = f"{cls_name} {conf_number:.2f}"  # 클래스 이름과 신뢰도

            # 클래스별 색상 선택
            box_color = color_dict.get(cls_name, (0, 255, 0))  # cls_name 기반으로 색상 선택

            # 바운딩 박스 그리기
            image = cv2.rectangle(image, (x1_int, y1_int), (x2_int, y2_int), box_color, 3)

            # 바운딩 박스의 중앙 좌표 계산
            center_box_x = int((x1_int + x2_int) / 2)
            center_box_y = int((y1_int + y2_int) / 2)

            # 이미지 중심과 바운딩 박스 중심 사이에서 바운딩 박스에 30% 더 가까운 지점 계산
            mid_x = int(center_image_x + 0.8 * (center_box_x - center_image_x))
            mid_y = int(center_image_y + 0.8 * (center_box_y - center_image_y))

            # 텍스트가 이미지 중심과 바운딩 박스 사이에서 30% 더 바운딩 박스에 가깝게 위치하도록 조정
            image = cv2.putText(image, label, (mid_x, mid_y), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)

        # 탐지된 객체 수를 출력하는 부분
        result_label.config(text=f"탐지된 객체 수: {len(final_boxes)}")

        # 이미지 표시
        show_image_from_array(image)

    except Exception as e:
        result_label.config(text=f"예측 중 오류 발생: {str(e)}")

# def predict_image(file_path):
#     try:
#         # 이미지 로드 및 YOLO 예측 수행
#         results = model.predict(file_path, save=False, imgsz=640, conf=0.2)

#         image = cv2.imread(file_path)
#         image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

#         # 각 클래스별 탐지된 개수를 저장할 임시 딕셔너리
#         current_detection_count = {key: 0 for key in custom_names.values()}
#         detection_details = []  # 탐지된 객체의 상세 정보를 저장할 리스트

#         final_boxes = []  # 최종 박스를 저장할 리스트

#         # 바운딩 박스 처리
#         for r in results:
#             r.names = custom_names

#             boxes = r.boxes.xyxy
#             cls = r.boxes.cls
#             conf = r.boxes.conf
#             cls_dict = r.names

#             # 바운딩 박스 및 클래스 이름 처리
#             for box, cls_number, conf_value in zip(boxes, cls, conf):
#                 x1, y1, x2, y2 = box
#                 x1_int, y1_int = int(x1.item()), int(y1.item())
#                 x2_int, y2_int = int(x2.item()), int(y2.item())
#                 cls_name = cls_dict[int(cls_number.item())]
#                 conf_number = float(conf_value.item())

#                 # 탐지된 객체 수 업데이트
#                 current_detection_count[cls_name] += 1
#                 detection_summary[cls_name] += 1

#                 # 총합 계산
#                 total_sums[cls_name] += 1

#                 # 탐지된 객체 정보 저장
#                 detection_details.append(f"{cls_name}: ({x1_int}, {y1_int})에서 ({x2_int}, {y2_int}) [확률: {conf_number:.2f}]")

#                 final_boxes.append({"coords": (x1_int, y1_int, x2_int, y2_int), 
#                                     "cls_name": cls_name, "conf": conf_number})

#         # 테이블에 결과 추가
#         new_data = (
#             os.path.basename(file_path),
#             current_detection_count['waist folding'],
#             current_detection_count['punching hole'],
#             current_detection_count['welding line'],
#             current_detection_count['crescent gap'],
#             current_detection_count['water spot'],
#             current_detection_count['oil spot'],
#             current_detection_count['silk spot'],
#             current_detection_count['inclusion'],
#             current_detection_count['rolled pit'],
#             current_detection_count['crease']
#         )

#         tree.insert("", tk.END, values=new_data)

#         # 최종적으로 남은 박스만 그리기
#         for box in final_boxes:
#             x1_int, y1_int, x2_int, y2_int = box["coords"]
#             cls_name = box["cls_name"]
#             conf_number = box["conf"]

#             label = f"{cls_name} {conf_number:.2f}"

#             # 클래스별 색상 선택
#             box_color = color_dict.get(cls_name, (0, 255, 0))  # cls_name 기반으로 색상 선택

#             # 바운딩 박스 그리기
#             image = cv2.rectangle(image, (x1_int, y1_int), (x2_int, y2_int), box_color, 2)
#             image = cv2.putText(image, label, (x1_int, y1_int - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)
            

#         # 탐지된 객체 수와 각 객체의 정보를 result_label에 업데이트
#         detection_summary_text = f"탐지된 오류 수: {len(final_boxes)}\n\n"
#         detection_summary_text += "\n".join(detection_details)  # 탐지된 객체 정보를 줄바꿈으로 나열
#         result_label.config(text=detection_summary_text, font=("Helvetica", 16))  # 폰트 크기 16으로 설정

#         # 이미지 표시
#         show_image_from_array(image)

#         # *** 예측된 이미지 저장 ***
#         save_image_path = os.path.join(save_path, f"output_{os.path.basename(file_path)}")
#         cv2.imwrite(save_image_path, cv2.cvtColor(image, cv2.COLOR_RGB2BGR))  # BGR로 변환하여 저장
#         print(f"이미지 저장됨: {save_image_path}")

#         # 총합 업데이트
#         update_summary()

#     except Exception as e:
#         result_label.config(text=f"예측 중 오류 발생: {str(e)}")


# 배열로부터 이미지를 GUI에 표시하는 함수 (OpenCV 이미지를 PIL 이미지로 변환)
def show_image_from_array(image_array):
    img = Image.fromarray(image_array)
    img = img.resize((320, 320))  # 이미지를 적절한 크기로 리사이즈
    img_tk = ImageTk.PhotoImage(img)
    image_label.config(image=img_tk)
    image_label.image = img_tk

# 폴더 선택 버튼
folder_button = tk.Button(window, text="폴더 선택 및 분석", command=select_folder)
folder_button.grid(row=4, column=0, pady=10)

# 파일 업로드 버튼 추가
upload_button = tk.Button(window, text="파일 업로드 및 분석", command=upload_and_predict)
upload_button.grid(row=4, column=1, pady=10)

# 총합을 보여줄 Treeview (하단에 위치)
summary_tree_frame = tk.Frame(window, height=16)  # 창 높이의 20% 설정
summary_tree_frame.grid(row=3, column=0, columnspan=2, padx=10, pady=10, sticky="nsew")

summary_tree = ttk.Treeview(summary_tree_frame, columns=('name', 'waist_folding', 'punching_hole', 
                                             'welding_line', 'crescent_gap', 'water_spot', 
                                             'oil_spot', 'silk_spot', 'inclusion', 
                                             'rolled_pit', 'crease'), show='headings', height=2)

summary_tree.heading("name", text='Total')
summary_tree.heading("waist_folding", text="waist_folding")
summary_tree.heading("punching_hole", text="punching_hole")
summary_tree.heading("welding_line", text="welding_line")
summary_tree.heading("crescent_gap", text="crescent_gap")
summary_tree.heading("water_spot", text="water_spot")
summary_tree.heading("oil_spot", text="oil_spot")
summary_tree.heading("silk_spot", text="silk_spot")
summary_tree.heading("inclusion", text="inclusion")
summary_tree.heading("rolled_pit", text="rolled_pit")
summary_tree.heading("crease", text="crease")

summary_tree.column("name", width=120, )
summary_tree.column("waist_folding", width=100)
summary_tree.column("punching_hole", width=100)
summary_tree.column("welding_line", width=100)
summary_tree.column("crescent_gap", width=100)
summary_tree.column("water_spot", width=100)
summary_tree.column("oil_spot", width=100)
summary_tree.column("silk_spot", width=100)
summary_tree.column("inclusion", width=100)
summary_tree.column("rolled_pit", width=100)
summary_tree.column("crease", width=100)

# 총합 테이블 배치
summary_tree.pack(fill=tk.BOTH, expand=True)

window.mainloop()



image 1/1 C:\KDT_SF6\vision_ai\  \img_01_424799300_01133.jpg: 320x640 1 2_hanfeng, 1 3_yueyawan, 32.1ms
Speed: 2.0ms preprocess, 32.1ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

image 1/1 C:\KDT_SF6\vision_ai\  \img_01_424799600_00001.jpg: 320x640 1 2_hanfeng, 1 3_yueyawan, 35.5ms
Speed: 2.0ms preprocess, 35.5ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)


In [4]:
import tkinter as tk
from tkinter import ttk, filedialog
from PIL import Image, ImageTk
from ultralytics import YOLO
import cv2
import os

# YOLO 모델 로드
model = YOLO('C:/KDT_SF6/vision_ai//best (1).pt')  # 모델 경로를 여기에 설정하세요

# 사용자 정의 클래스 이름
custom_names = {0: 'yaozhe', 1: 'waist folding', 2: 'punching hole', 3: 'welding line', 
                4: 'crescent gap', 5: 'water spot', 6: 'oil spot', 7: 'silk spot', 
                8: 'inclusion', 9: 'rolled pit', 10: 'crease', 11: 'd'}

# 클래스별 색상 설정
color_dict = {
    0: (255, 0, 0),   # yaozhe - 빨강
    1: (0, 255, 0),   # waist folding - 초록
    2: (0, 0, 255),   # punching hole - 파랑
    # 나머지 클래스 색상 추가
}

# 이미지 저장 경로 설정
save_path = 'C:/KDT_SF6/vision_ai'  # 저장할 폴더 경로

# 총합을 저장할 딕셔너리 (초기값은 0)
total_sums = {key: 0 for key in custom_names.values()}

# 탐지 결과를 저장할 딕셔너리 (초기값 0)
detection_summary = {key: 0 for key in custom_names.values()}  # 각 클래스별 탐지 개수 초기화

# GUI 초기 설정
window = tk.Tk()
window.title("YOLO 이미지 탐지")
window.geometry("1200x800")

# 이미지 표시를 위한 라벨 추가 (좌측에 이미지 배치)
image_label = tk.Label(window)
image_label.grid(row=0, column=0, padx=10, pady=10, sticky="nw")

# 결과 레이블 (사진의 우측에 배치)
result_label = tk.Label(window, text="")
result_label.grid(row=0, column=1, padx=10, pady=10, sticky="nw")  # 이미지의 바로 우측에 배치

# Treeview 생성 및 열 설정 (하단에 위치)
tree_frame = tk.Frame(window)
tree_frame.grid(row=2, column=0, columnspan=2, sticky="nsew", padx=10, pady=10)

# 스크롤바 생성
tree_scroll = tk.Scrollbar(tree_frame)
tree_scroll.pack(side=tk.RIGHT, fill=tk.Y)

tree = ttk.Treeview(tree_frame, columns=('name', 'waist_folding', 'punching_hole', 
                                         'welding_line', 'crescent_gap', 'water_spot', 
                                         'oil_spot', 'silk_spot', 'inclusion', 
                                         'rolled_pit', 'crease'), show='headings',
                    yscrollcommand=tree_scroll.set)

# 스크롤바에 트리뷰 연결
tree_scroll.config(command=tree.yview)

# Treeview 열 제목 설정
tree.heading("name", text='name')
tree.heading("waist_folding", text="waist_folding")
tree.heading("punching_hole", text="punching_hole")
tree.heading("welding_line", text="welding_line")
tree.heading("crescent_gap", text="crescent_gap")
tree.heading("water_spot", text="water_spot")
tree.heading("oil_spot", text="oil_spot")
tree.heading("silk_spot", text="silk_spot")
tree.heading("inclusion", text="inclusion")
tree.heading("rolled_pit", text="rolled_pit")
tree.heading("crease", text="crease")

# Treeview 열 너비 설정
tree.column("name", width=120)
tree.column("waist_folding", width=100)
tree.column("punching_hole", width=100)
tree.column("welding_line", width=100)
tree.column("crescent_gap", width=100)
tree.column("water_spot", width=100)
tree.column("oil_spot", width=100)
tree.column("silk_spot", width=100)
tree.column("inclusion", width=100)
tree.column("rolled_pit", width=100)
tree.column("crease", width=100)

# Treeview 배치
tree.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)

# 총합 업데이트 함수
def update_summary():
    summary_tree.delete(*summary_tree.get_children())  # 기존 항목 삭제
    new_data = (
        "Total",
        total_sums['waist folding'],
        total_sums['punching hole'],
        total_sums['welding line'],
        total_sums['crescent gap'],
        total_sums['water spot'],
        total_sums['oil spot'],
        total_sums['silk spot'],
        total_sums['inclusion'],
        total_sums['rolled pit'],
        total_sums['crease']
    )
    summary_tree.insert("", tk.END, values=new_data)  # 총합 업데이트

# 파일을 순차적으로 확인하여 예측을 수행하는 함수
def check_for_new_image():
    global file_list, file_index
    if file_list and file_index < len(file_list):
        file_path = os.path.join(selected_folder, file_list[file_index])
        predict_image(file_path)
        file_index += 1

    # 더 이상 파일이 없으면 종료
    if file_index >= len(file_list):
        result_label.config(text="모든 파일을 처리하였습니다.")
        return

    # 2초마다 이 함수를 다시 호출
    window.after(10000, check_for_new_image)

# 폴더 선택 및 이미지 처리 함수
def select_folder():
    global file_list, file_index, selected_folder
    selected_folder = filedialog.askdirectory()  # 폴더 선택 다이얼로그
    if selected_folder:
        file_list = [f for f in os.listdir(selected_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]
        file_index = 0
        if file_list:
            check_for_new_image()

# 개별 파일 선택 및 예측 수행 함수
def predict_image(file_path):
    try:
        # 이미지 로드 및 YOLO 예측 수행
        results = model.predict(file_path, save=False, imgsz=640, conf=0.2)

        image = cv2.imread(file_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # 각 클래스별 탐지된 개수를 저장할 임시 딕셔너리
        current_detection_count = {key: 0 for key in custom_names.values()}
        detection_details = []  # 탐지된 객체의 상세 정보를 저장할 리스트

        final_boxes = []  # 최종 박스를 저장할 리스트

        # 바운딩 박스 처리
        for r in results:
            r.names = custom_names

            boxes = r.boxes.xyxy
            cls = r.boxes.cls
            conf = r.boxes.conf
            cls_dict = r.names

            # 바운딩 박스 및 클래스 이름 처리
            for box, cls_number, conf_value in zip(boxes, cls, conf):
                x1, y1, x2, y2 = box
                x1_int, y1_int = int(x1.item()), int(y1.item())
                x2_int, y2_int = int(x2.item()), int(y2.item())
                cls_name = cls_dict[int(cls_number.item())]
                conf_number = float(conf_value.item())

                # 탐지된 객체 수 업데이트
                current_detection_count[cls_name] += 1
                detection_summary[cls_name] += 1

                # 총합 계산
                total_sums[cls_name] += 1

                # 탐지된 객체 정보 저장
                detection_details.append(f"{cls_name}: ({x1_int}, {y1_int})에서 ({x2_int}, {y2_int}) [확률: {conf_number:.2f}]")

                final_boxes.append({"coords": (x1_int, y1_int, x2_int, y2_int), 
                                    "cls_name": cls_name, "conf": conf_number})

        # 테이블에 결과 추가
        new_data = (
            os.path.basename(file_path),
            current_detection_count['waist folding'],
            current_detection_count['punching hole'],
            current_detection_count['welding line'],
            current_detection_count['crescent gap'],
            current_detection_count['water spot'],
            current_detection_count['oil spot'],
            current_detection_count['silk spot'],
            current_detection_count['inclusion'],
            current_detection_count['rolled pit'],
            current_detection_count['crease']
        )

        tree.insert("", tk.END, values=new_data)

        # 최종적으로 남은 박스만 그리기
        for box in final_boxes:
            x1_int, y1_int, x2_int, y2_int = box["coords"]
            cls_name = box["cls_name"]
            conf_number = box["conf"]

            label = f"{cls_name} {conf_number:.2f}"

            # 클래스별 색상 선택
            box_color = color_dict.get(cls_name, (0, 255, 0))  # cls_name 기반으로 색상 선택

            # 바운딩 박스 그리기
            image = cv2.rectangle(image, (x1_int, y1_int), (x2_int, y2_int), box_color, 4)
            image = cv2.putText(image, label, (x1_int, y1_int - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 0, 0), 2)

        # 탐지된 객체 수와 각 객체의 정보를 result_label에 업데이트
        detection_summary_text = f"탐지된 오류 수: {len(final_boxes)}\n\n"
        detection_summary_text += "\n".join(detection_details)  # 탐지된 객체 정보를 줄바꿈으로 나열
        result_label.config(text=detection_summary_text, font=("Helvetica", 16))  # 폰트 크기 16으로 설정

        # 이미지 표시
        show_image_from_array(image)

        # *** 예측된 이미지 저장 ***
        save_image_path = os.path.join(save_path, f"output_{os.path.basename(file_path)}")
        cv2.imwrite(save_image_path, cv2.cvtColor(image, cv2.COLOR_RGB2BGR))  # BGR로 변환하여 저장
        print(f"이미지 저장됨: {save_image_path}")

        # 총합 업데이트
        update_summary()

    except Exception as e:
        result_label.config(text=f"예측 중 오류 발생: {str(e)}")

# 배열로부터 이미지를 GUI에 표시하는 함수 (OpenCV 이미지를 PIL 이미지로 변환)
def show_image_from_array(image_array):
    img = Image.fromarray(image_array)
    img = img.resize((320, 320))  # 이미지를 적절한 크기로 리사이즈
    img_tk = ImageTk.PhotoImage(img)
    image_label.config(image=img_tk)
    image_label.image = img_tk

# 총합을 보여줄 Treeview (하단에 위치)
summary_tree_frame = tk.Frame(window, height=16)  # 창 높이의 20% 설정
summary_tree_frame.grid(row=3, column=0, columnspan=2, padx=10, pady=10, sticky="nsew")

summary_tree = ttk.Treeview(summary_tree_frame, columns=('name', 'waist_folding', 'punching_hole', 
                                             'welding_line', 'crescent_gap', 'water_spot', 
                                             'oil_spot', 'silk_spot', 'inclusion', 
                                             'rolled_pit', 'crease'), show='headings', height=2)

summary_tree.heading("name", text='Total')
summary_tree.heading("waist_folding", text="waist_folding")
summary_tree.heading("punching_hole", text="punching_hole")
summary_tree.heading("welding_line", text="welding_line")
summary_tree.heading("crescent_gap", text="crescent_gap")
summary_tree.heading("water_spot", text="water_spot")
summary_tree.heading("oil_spot", text="oil_spot")
summary_tree.heading("silk_spot", text="silk_spot")
summary_tree.heading("inclusion", text="inclusion")
summary_tree.heading("rolled_pit", text="rolled_pit")
summary_tree.heading("crease", text="crease")

summary_tree.column("name", width=120, )
summary_tree.column("waist_folding", width=100)
summary_tree.column("punching_hole", width=100)
summary_tree.column("welding_line", width=100)
summary_tree.column("crescent_gap", width=100)
summary_tree.column("water_spot", width=100)
summary_tree.column("oil_spot", width=100)
summary_tree.column("silk_spot", width=100)
summary_tree.column("inclusion", width=100)
summary_tree.column("rolled_pit", width=100)
summary_tree.column("crease", width=100)

# 총합 테이블 배치
summary_tree.pack(fill=tk.BOTH, expand=True)

# 프로그램 시작 시 폴더 선택 창 띄우기
select_folder()

window.mainloop()



image 1/1 C:\KDT_SF6\vision_ai\  \img_01_424799300_01133.jpg: 320x640 1 2_hanfeng, 1 3_yueyawan, 35.7ms
Speed: 2.0ms preprocess, 35.7ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)
이미지 저장됨: C:/KDT_SF6/vision_ai\output_img_01_424799300_01133.jpg

image 1/1 C:\KDT_SF6\vision_ai\  \img_01_424799600_00001.jpg: 320x640 1 2_hanfeng, 1 3_yueyawan, 37.0ms
Speed: 2.0ms preprocess, 37.0ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)
이미지 저장됨: C:/KDT_SF6/vision_ai\output_img_01_424799600_00001.jpg


## 폴더 미리 설정

In [20]:
import tkinter as tk
from tkinter import ttk
from PIL import Image, ImageTk
from ultralytics import YOLO
import cv2
import os

# YOLO 모델 로드
model = YOLO('C:/KDT_SF6/vision_ai/best (1).pt')  # 모델 경로를 여기에 설정하세요

# 사용자 정의 클래스 이름
custom_names = {0: 'yaozhe', 1: 'waist folding', 2: 'punching hole', 3: 'welding line', 
                4: 'crescent gap', 5: 'water spot', 6: 'oil spot', 7: 'silk spot', 
                8: 'inclusion', 9: 'rolled pit', 10: 'crease', 11: 'd'}

# 클래스별 색상 설정
color_dict = {
    0: (255, 0, 0),   # yaozhe - 빨강
    1: (0, 255, 0),   # waist folding - 초록
    2: (0, 0, 255),   # punching hole - 파랑
    # 나머지 클래스 색상 추가
}

# 이미지 저장 경로 설정
save_path = 'C:/KDT_SF6/vision_ai'  # 저장할 폴더 경로
image_folder = 'C:/KDT_SF6/vision_ai/새 폴더'  # 고정된 폴더 경로

# 총합을 저장할 딕셔너리 (초기값은 0)
total_sums = {key: 0 for key in custom_names.values()}

# 탐지 결과를 저장할 딕셔너리 (초기값 0)
detection_summary = {key: 0 for key in custom_names.values()}  # 각 클래스별 탐지 개수 초기화

# GUI 초기 설정
window = tk.Tk()
window.title("YOLO 이미지 탐지")
window.geometry("1200x800")

# 이미지 표시를 위한 라벨 추가 (좌측에 이미지 배치)
image_label = tk.Label(window)
image_label.grid(row=0, column=0, padx=10, pady=10, sticky="nw")

# 결과 레이블 (사진의 우측에 배치)
result_label = tk.Label(window, text="")
result_label.grid(row=0, column=1, padx=10, pady=10, sticky="nw")  # 이미지의 바로 우측에 배치

# Treeview 생성 및 열 설정 (하단에 위치)
tree_frame = tk.Frame(window)
tree_frame.grid(row=2, column=0, columnspan=2, sticky="nsew", padx=10, pady=10)

# 스크롤바 생성
tree_scroll = tk.Scrollbar(tree_frame)
tree_scroll.pack(side=tk.RIGHT, fill=tk.Y)

tree = ttk.Treeview(tree_frame, columns=('name', 'waist_folding', 'punching_hole', 
                                         'welding_line', 'crescent_gap', 'water_spot', 
                                         'oil_spot', 'silk_spot', 'inclusion', 
                                         'rolled_pit', 'crease'), show='headings',
                    yscrollcommand=tree_scroll.set)

# 스크롤바에 트리뷰 연결
tree_scroll.config(command=tree.yview)

# Treeview 열 제목 설정
tree.heading("name", text='name')
tree.heading("waist_folding", text="waist_folding")
tree.heading("punching_hole", text="punching_hole")
tree.heading("welding_line", text="welding_line")
tree.heading("crescent_gap", text="crescent_gap")
tree.heading("water_spot", text="water_spot")
tree.heading("oil_spot", text="oil_spot")
tree.heading("silk_spot", text="silk_spot")
tree.heading("inclusion", text="inclusion")
tree.heading("rolled_pit", text="rolled_pit")
tree.heading("crease", text="crease")

# Treeview 열 너비 설정
tree.column("name", width=120)
tree.column("waist_folding", width=100)
tree.column("punching_hole", width=100)
tree.column("welding_line", width=100)
tree.column("crescent_gap", width=100)
tree.column("water_spot", width=100)
tree.column("oil_spot", width=100)
tree.column("silk_spot", width=100)
tree.column("inclusion", width=100)
tree.column("rolled_pit", width=100)
tree.column("crease", width=100)

# Treeview 배치
tree.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)

# 총합 업데이트 함수
def update_summary():
    summary_tree.delete(summary_tree.get_children())  # 기존 항목 삭제
    new_data = (
        "Total",
        total_sums['waist folding'],
        total_sums['punching hole'],
        total_sums['welding line'],
        total_sums['crescent gap'],
        total_sums['water spot'],
        total_sums['oil spot'],
        total_sums['silk spot'],
        total_sums['inclusion'],
        total_sums['rolled pit'],
        total_sums['crease']
    )
    summary_tree.insert("", tk.END, values=new_data)  # 총합 업데이트

# 파일을 순차적으로 확인하여 예측을 수행하는 함수
def check_for_new_image():
    global file_list, file_index
    if file_list and file_index < len(file_list):
        file_path = os.path.join(image_folder, file_list[file_index])
        predict_image(file_path)
        file_index += 1

    # 더 이상 파일이 없으면 종료
    if file_index >= len(file_list):
        result_label.config(text="모든 파일을 처리하였습니다.")
        return

    # 2초마다 이 함수를 다시 호출
    window.after(2000, check_for_new_image)

# 개별 파일 선택 및 예측 수행 함수
def predict_image(file_path):
    try:
        # 이미지 로드 및 YOLO 예측 수행
        results = model.predict(file_path, save=False, imgsz=640, conf=0.2)

        image = cv2.imread(file_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # 각 클래스별 탐지된 개수를 저장할 임시 딕셔너리
        current_detection_count = {key: 0 for key in custom_names.values()}
        detection_details = []  # 탐지된 객체의 상세 정보를 저장할 리스트

        final_boxes = []  # 최종 박스를 저장할 리스트

        # 바운딩 박스 처리
        for r in results:
            r.names = custom_names

            boxes = r.boxes.xyxy
            cls = r.boxes.cls
            conf = r.boxes.conf
            cls_dict = r.names

            # 바운딩 박스 및 클래스 이름 처리
            for box, cls_number, conf_value in zip(boxes, cls, conf):
                x1, y1, x2, y2 = box
                x1_int, y1_int = int(x1.item()), int(y1.item())
                x2_int, y2_int = int(x2.item()), int(y2.item())
                cls_name = cls_dict[int(cls_number.item())]
                conf_number = float(conf_value.item())

                # 탐지된 객체 수 업데이트
                current_detection_count[cls_name] += 1
                detection_summary[cls_name] += 1

                # 총합 계산
                total_sums[cls_name] += 1

                # 탐지된 객체 정보 저장
                detection_details.append(f"{cls_name}: ({x1_int}, {y1_int})에서 ({x2_int}, {y2_int}) [확률: {conf_number:.2f}]")

                final_boxes.append({"coords": (x1_int, y1_int, x2_int, y2_int), 
                                    "cls_name": cls_name, "conf": conf_number})

        # 테이블에 결과 추가
        new_data = (
            os.path.basename(file_path),
            current_detection_count['waist folding'],
            current_detection_count['punching hole'],
            current_detection_count['welding line'],
            current_detection_count['crescent gap'],
            current_detection_count['water spot'],
            current_detection_count['oil spot'],
            current_detection_count['silk spot'],
            current_detection_count['inclusion'],
            current_detection_count['rolled pit'],
            current_detection_count['crease']
        )

        tree.insert("", tk.END, values=new_data)

        # 최종적으로 남은 박스만 그리기
        for box in final_boxes:
            x1_int, y1_int, x2_int, y2_int = box["coords"]
            cls_name = box["cls_name"]
            conf_number = box["conf"]

            label = f"{cls_name} {conf_number:.2f}"

            # 클래스별 색상 선택
            box_color = color_dict.get(cls_name, (0, 255, 0))  # cls_name 기반으로 색상 선택

            # 바운딩 박스 그리기
            image = cv2.rectangle(image, (x1_int, y1_int), (x2_int, y2_int), box_color, 4)
            image = cv2.putText(image, label, (x1_int, y1_int - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 0, 0), 2)

        # 탐지된 객체 수와 각 객체의 정보를 result_label에 업데이트
        detection_summary_text = f"탐지된 오류 수: {len(final_boxes)}\n\n"
        detection_summary_text += "\n".join(detection_details)  # 탐지된 객체 정보를 줄바꿈으로 나열
        result_label.config(text=detection_summary_text, font=("Helvetica", 16))  # 폰트 크기 16으로 설정

        # 이미지 표시
        show_image_from_array(image)

        # *** 예측된 이미지 저장 ***
        save_image_path = os.path.join(save_path, f"output_{os.path.basename(file_path)}")
        cv2.imwrite(save_image_path, cv2.cvtColor(image, cv2.COLOR_RGB2BGR))  # BGR로 변환하여 저장
        print(f"이미지 저장됨: {save_image_path}")

        # 총합 업데이트
        update_summary()

    except Exception as e:
        result_label.config(text=f"예측 중 오류 발생: {str(e)}")

# 배열로부터 이미지를 GUI에 표시하는 함수 (OpenCV 이미지를 PIL 이미지로 변환)
def show_image_from_array(image_array):
    img = Image.fromarray(image_array)
    img = img.resize((320, 320))  # 이미지를 적절한 크기로 리사이즈
    img_tk = ImageTk.PhotoImage(img)
    image_label.config(image=img_tk)
    image_label.image = img_tk

# 총합을 보여줄 Treeview (하단에 위치)
summary_tree_frame = tk.Frame(window, height=16)  # 창 높이의 20% 설정
summary_tree_frame.grid(row=3, column=0, columnspan=2, padx=10, pady=10, sticky="nsew")

summary_tree = ttk.Treeview(summary_tree_frame, columns=('name', 'waist_folding', 'punching_hole', 
                                             'welding_line', 'crescent_gap', 'water_spot', 
                                             'oil_spot', 'silk_spot', 'inclusion', 
                                             'rolled_pit', 'crease'), show='headings', height=2)

summary_tree.heading("name", text='Total')
summary_tree.heading("waist_folding", text="waist_folding")
summary_tree.heading("punching_hole", text="punching_hole")
summary_tree.heading("welding_line", text="welding_line")
summary_tree.heading("crescent_gap", text="crescent_gap")
summary_tree.heading("water_spot", text="water_spot")
summary_tree.heading("oil_spot", text="oil_spot")
summary_tree.heading("silk_spot", text="silk_spot")
summary_tree.heading("inclusion", text="inclusion")
summary_tree.heading("rolled_pit", text="rolled_pit")
summary_tree.heading("crease", text="crease")

summary_tree.column("name", width=120, )
summary_tree.column("waist_folding", width=100)
summary_tree.column("punching_hole", width=100)
summary_tree.column("welding_line", width=100)
summary_tree.column("crescent_gap", width=100)
summary_tree.column("water_spot", width=100)
summary_tree.column("oil_spot", width=100)
summary_tree.column("silk_spot", width=100)
summary_tree.column("inclusion", width=100)
summary_tree.column("rolled_pit", width=100)
summary_tree.column("crease", width=100)

# 총합 테이블 배치
summary_tree.pack(fill=tk.BOTH, expand=True)

# 고정된 폴더에서 이미지 파일 목록 가져오기
file_list = [f for f in os.listdir(image_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]
file_index = 0

# 프로그램 시작 시 바로 이미지 처리 시작
check_for_new_image()

window.mainloop()



image 1/1 C:\KDT_SF6\vision_ai\ \img_01_424799600_00001.jpg: 320x640 1 2_hanfeng, 1 3_yueyawan, 35.3ms
Speed: 1.0ms preprocess, 35.3ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)
이미지 저장됨: C:/KDT_SF6/vision_ai\output_img_01_424799600_00001.jpg

image 1/1 C:\KDT_SF6\vision_ai\ \img_02_3402618000_00002.jpg: 320x640 1 10_yaozhed, 33.6ms
Speed: 1.2ms preprocess, 33.6ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)
이미지 저장됨: C:/KDT_SF6/vision_ai\output_img_02_3402618000_00002.jpg


## 파일 폴더 정보 완성

In [10]:
import tkinter as tk
from tkinter import ttk, filedialog
from PIL import Image, ImageTk
from ultralytics import YOLO
import cv2
import os

# YOLO 모델 로드
model = YOLO('C:/KDT_SF6/vision_ai/best (1).pt')  # 모델 경로를 여기에 설정하세요

# 사용자 정의 클래스 이름
custom_names = {0: 'yaozhe', 1: 'waist folding', 2: 'punching hole', 3: 'welding line', 
                4: 'crescent gap', 5: 'water spot', 6: 'oil spot', 7: 'silk spot', 
                8: 'inclusion', 9: 'rolled pit', 10: 'crease', 11: 'd'}

# 클래스별 색상 설정
color_dict = {
    0: (255, 0, 0),   # yaozhe - 빨강
    1: (0, 255, 0),   # waist folding - 초록
    2: (0, 0, 255),   # punching hole - 파랑
    # 나머지 클래스 색상 추가
}

# 이미지 저장 경로 설정
save_path = 'C:/KDT_SF6/vision_ai'  # 저장할 폴더 경로

# 총합을 저장할 딕셔너리 (초기값은 0)
total_sums = {key: 0 for key in custom_names.values()}

# 탐지 결과를 저장할 딕셔너리 (초기값 0)
detection_summary = {key: 0 for key in custom_names.values()}  # 각 클래스별 탐지 개수 초기화

# GUI 초기 설정
window = tk.Tk()
window.title("YOLO 이미지 탐지")
window.geometry("1200x800")

# 이미지 표시를 위한 라벨 추가 (좌측에 이미지 배치)
image_label = tk.Label(window)
image_label.grid(row=0, column=0, padx=10, pady=10, sticky="nw")

# 결과 레이블 (사진의 우측에 배치)
result_label = tk.Label(window, text="")
result_label.grid(row=0, column=1, padx=10, pady=10, sticky="nw")  # 이미지의 바로 우측에 배치

# Treeview 생성 및 열 설정 (하단에 위치)
tree_frame = tk.Frame(window)
tree_frame.grid(row=2, column=0, columnspan=2, sticky="nsew", padx=10, pady=10)

# 스크롤바 생성
tree_scroll = tk.Scrollbar(tree_frame)
tree_scroll.pack(side=tk.RIGHT, fill=tk.Y)

tree = ttk.Treeview(tree_frame, columns=('name', 'waist_folding', 'punching_hole', 
                                         'welding_line', 'crescent_gap', 'water_spot', 
                                         'oil_spot', 'silk_spot', 'inclusion', 
                                         'rolled_pit', 'crease'), show='headings',
                    yscrollcommand=tree_scroll.set)

# 스크롤바에 트리뷰 연결
tree_scroll.config(command=tree.yview)

# Treeview 열 제목 설정
tree.heading("name", text='name')
tree.heading("waist_folding", text="waist_folding")
tree.heading("punching_hole", text="punching_hole")
tree.heading("welding_line", text="welding_line")
tree.heading("crescent_gap", text="crescent_gap")
tree.heading("water_spot", text="water_spot")
tree.heading("oil_spot", text="oil_spot")
tree.heading("silk_spot", text="silk_spot")
tree.heading("inclusion", text="inclusion")
tree.heading("rolled_pit", text="rolled_pit")
tree.heading("crease", text="crease")

# Treeview 열 너비 설정
tree.column("name", width=120)
tree.column("waist_folding", width=100)
tree.column("punching_hole", width=100)
tree.column("welding_line", width=100)
tree.column("crescent_gap", width=100)
tree.column("water_spot", width=100)
tree.column("oil_spot", width=100)
tree.column("silk_spot", width=100)
tree.column("inclusion", width=100)
tree.column("rolled_pit", width=100)
tree.column("crease", width=100)

# Treeview 배치
tree.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)

# 총합 업데이트 함수
def update_summary():
    summary_tree.delete(*summary_tree.get_children())  # 기존 항목 삭제
    new_data = (
        "Total",
        total_sums['waist folding'],
        total_sums['punching hole'],
        total_sums['welding line'],
        total_sums['crescent gap'],
        total_sums['water spot'],
        total_sums['oil spot'],
        total_sums['silk spot'],
        total_sums['inclusion'],
        total_sums['rolled pit'],
        total_sums['crease']
    )
    summary_tree.insert("", tk.END, values=new_data)  # 총합 업데이트

# 파일을 순차적으로 확인하여 예측을 수행하는 함수
def check_for_new_image():
    global file_list, file_index
    if file_list and file_index < len(file_list):
        file_path = os.path.join(selected_folder, file_list[file_index])
        predict_image(file_path)
        file_index += 1

    # 더 이상 파일이 없으면 종료
    if file_index >= len(file_list):
        result_label.config(text="모든 파일을 처리하였습니다.")
        return

    # 2초마다 이 함수를 다시 호출
    window.after(2000, check_for_new_image)

# 폴더 선택 및 이미지 처리 함수
def select_folder():
    global file_list, file_index, selected_folder
    selected_folder = filedialog.askdirectory()  # 폴더 선택 다이얼로그
    if selected_folder:
        file_list = [f for f in os.listdir(selected_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]
        file_index = 0
        if file_list:
            check_for_new_image()

# 개별 파일 선택 및 예측 수행 함수
def upload_and_predict():
    file_path = filedialog.askopenfilename(title="이미지 파일 선택", filetypes=[("Image Files", "*.png;*.jpg;*.jpeg")])
    if file_path:
        predict_image(file_path)

def predict_image(file_path):
    try:
        # 이미지 로드 및 YOLO 예측 수행
        results = model.predict(file_path, save=False, imgsz=640, conf=0.2)

        image = cv2.imread(file_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # 각 클래스별 탐지된 개수를 저장할 임시 딕셔너리
        current_detection_count = {key: 0 for key in custom_names.values()}
        detection_details = []  # 탐지된 객체의 상세 정보를 저장할 리스트

        final_boxes = []  # 최종 박스를 저장할 리스트

        # 바운딩 박스 처리
        for r in results:
            r.names = custom_names

            boxes = r.boxes.xyxy
            cls = r.boxes.cls
            conf = r.boxes.conf
            cls_dict = r.names

            # 바운딩 박스 및 클래스 이름 처리
            for box, cls_number, conf_value in zip(boxes, cls, conf):
                x1, y1, x2, y2 = box
                x1_int, y1_int = int(x1.item()), int(y1.item())
                x2_int, y2_int = int(x2.item()), int(y2.item())
                cls_name = cls_dict[int(cls_number.item())]
                conf_number = float(conf_value.item())

                # 탐지된 객체 수 업데이트
                current_detection_count[cls_name] += 1
                detection_summary[cls_name] += 1

                # 총합 계산
                total_sums[cls_name] += 1

                # 탐지된 객체 정보 저장
                detection_details.append(f"{cls_name}: ({x1_int}, {y1_int})에서 ({x2_int}, {y2_int}) [확률: {conf_number:.2f}]")

                final_boxes.append({"coords": (x1_int, y1_int, x2_int, y2_int), 
                                    "cls_name": cls_name, "conf": conf_number})

        # 테이블에 결과 추가
        new_data = (
            os.path.basename(file_path),
            current_detection_count['waist folding'],
            current_detection_count['punching hole'],
            current_detection_count['welding line'],
            current_detection_count['crescent gap'],
            current_detection_count['water spot'],
            current_detection_count['oil spot'],
            current_detection_count['silk spot'],
            current_detection_count['inclusion'],
            current_detection_count['rolled pit'],
            current_detection_count['crease']
        )

        tree.insert("", tk.END, values=new_data)

        # 최종적으로 남은 박스만 그리기
        for box in final_boxes:
            x1_int, y1_int, x2_int, y2_int = box["coords"]
            cls_name = box["cls_name"]
            conf_number = box["conf"]

            label = f"{cls_name} {conf_number:.2f}"

            # 클래스별 색상 선택
            box_color = color_dict.get(cls_name, (0, 255, 0))  # cls_name 기반으로 색상 선택

            # 바운딩 박스 그리기
            image = cv2.rectangle(image, (x1_int, y1_int), (x2_int, y2_int), box_color, 2)
            image = cv2.putText(image, label, (x1_int, y1_int - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)

        # 탐지된 객체 수와 각 객체의 정보를 result_label에 업데이트
        detection_summary_text = f"탐지된 오류 수: {len(final_boxes)}\n\n"
        detection_summary_text += "\n".join(detection_details)  # 탐지된 객체 정보를 줄바꿈으로 나열
        result_label.config(text=detection_summary_text, font=("Helvetica", 16))  # 폰트 크기 16으로 설정

        # 이미지 표시
        show_image_from_array(image)

        # *** 예측된 이미지 저장 ***
        save_image_path = os.path.join(save_path, f"output_{os.path.basename(file_path)}")
        cv2.imwrite(save_image_path, cv2.cvtColor(image, cv2.COLOR_RGB2BGR))  # BGR로 변환하여 저장
        print(f"이미지 저장됨: {save_image_path}")

        # 총합 업데이트
        update_summary()

    except Exception as e:
        result_label.config(text=f"예측 중 오류 발생: {str(e)}")

# 배열로부터 이미지를 GUI에 표시하는 함수 (OpenCV 이미지를 PIL 이미지로 변환)
def show_image_from_array(image_array):
    img = Image.fromarray(image_array)
    img = img.resize((320, 320))  # 이미지를 적절한 크기로 리사이즈
    img_tk = ImageTk.PhotoImage(img)
    image_label.config(image=img_tk)
    image_label.image = img_tk

# 폴더 선택 버튼
folder_button = tk.Button(window, text="폴더 선택 및 분석", command=select_folder)
folder_button.grid(row=4, column=0, pady=10)

# 파일 업로드 버튼 추가
upload_button = tk.Button(window, text="파일 업로드 및 분석", command=upload_and_predict)
upload_button.grid(row=4, column=1, pady=10)

# 총합을 보여줄 Treeview (하단에 위치)
summary_tree_frame = tk.Frame(window, height=16)  # 창 높이의 20% 설정
summary_tree_frame.grid(row=3, column=0, columnspan=2, padx=10, pady=10, sticky="nsew")

summary_tree = ttk.Treeview(summary_tree_frame, columns=('name', 'waist_folding', 'punching_hole', 
                                             'welding_line', 'crescent_gap', 'water_spot', 
                                             'oil_spot', 'silk_spot', 'inclusion', 
                                             'rolled_pit', 'crease'), show='headings', height=2)

summary_tree.heading("name", text='Total')
summary_tree.heading("waist_folding", text="waist_folding")
summary_tree.heading("punching_hole", text="punching_hole")
summary_tree.heading("welding_line", text="welding_line")
summary_tree.heading("crescent_gap", text="crescent_gap")
summary_tree.heading("water_spot", text="water_spot")
summary_tree.heading("oil_spot", text="oil_spot")
summary_tree.heading("silk_spot", text="silk_spot")
summary_tree.heading("inclusion", text="inclusion")
summary_tree.heading("rolled_pit", text="rolled_pit")
summary_tree.heading("crease", text="crease")

summary_tree.column("name", width=120, )
summary_tree.column("waist_folding", width=100)
summary_tree.column("punching_hole", width=100)
summary_tree.column("welding_line", width=100)
summary_tree.column("crescent_gap", width=100)
summary_tree.column("water_spot", width=100)
summary_tree.column("oil_spot", width=100)
summary_tree.column("silk_spot", width=100)
summary_tree.column("inclusion", width=100)
summary_tree.column("rolled_pit", width=100)
summary_tree.column("crease", width=100)

# 총합 테이블 배치
summary_tree.pack(fill=tk.BOTH, expand=True)

window.mainloop()



image 1/1 C:\KDT_SF6\vision_ai\  \img_01_424799300_01133.jpg: 320x640 1 2_hanfeng, 1 3_yueyawan, 36.9ms
Speed: 2.0ms preprocess, 36.9ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)
이미지 저장됨: C:/KDT_SF6/vision_ai\output_img_01_424799300_01133.jpg

image 1/1 C:\KDT_SF6\vision_ai\  \img_01_424799600_00001.jpg: 320x640 1 2_hanfeng, 1 3_yueyawan, 28.2ms
Speed: 1.0ms preprocess, 28.2ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)
이미지 저장됨: C:/KDT_SF6/vision_ai\output_img_01_424799600_00001.jpg


In [12]:
import tkinter as tk
from tkinter import ttk, filedialog
from PIL import Image, ImageTk
import cv2
import numpy as np
import os
from tensorflow import keras

# 사용자 정의 클래스 이름
custom_names = {0: 'yaozhe', 1: 'waist folding', 2: 'punching hole', 3: 'welding line', 
                4: 'crescent gap', 5: 'water spot', 6: 'oil spot', 7: 'silk spot', 
                8: 'inclusion', 9: 'rolled pit', 10: 'crease', 11: 'd'}

# 저장할 이미지 경로 설정
save_path = 'C:/KDT_SF6/vision_ai/모델/개별모델'

# Keras 모델 로드 (결함별로 여러 모델 사용)
defect_classes = ['crease', 'crescent_gap', 'oil_spot', 'rolled_pit', 'punching_hole', 
                  'silk_spot', 'inclusion', 'waist folding', 'water_spot', 'welding_line']
models = {}

for defect in defect_classes:
    model_path = f"C:/KDT_SF6/vision_ai/drive-download-20240926T163114Z-001/class_location_test_{defect}.keras"
    if os.path.exists(model_path):
        models[defect] = keras.models.load_model(model_path, custom_objects={'mse': 'mean_squared_error'})
        print(f"모델 로드 성공: {defect}")
    else:
        print(f"모델 파일을 찾을 수 없습니다: {model_path}")

# GUI 초기 설정
window = tk.Tk()
window.title("Keras 이미지 탐지")
window.geometry("1200x800")

# 이미지 표시를 위한 라벨 추가 (좌측에 이미지 배치)
image_label = tk.Label(window)
image_label.grid(row=0, column=0, padx=10, pady=10, sticky="nw")

# 결과 레이블 (사진의 우측에 배치)
result_label = tk.Label(window, text="")
result_label.grid(row=0, column=1, padx=10, pady=10, sticky="nw")

# Treeview 생성 및 열 설정 (하단에 위치)
tree_frame = tk.Frame(window)
tree_frame.grid(row=2, column=0, columnspan=2, sticky="nsew", padx=10, pady=10)

# 스크롤바 생성
tree_scroll = tk.Scrollbar(tree_frame)
tree_scroll.pack(side=tk.RIGHT, fill=tk.Y)

tree = ttk.Treeview(tree_frame, columns=('name', 'waist_folding', 'punching_hole', 
                                         'welding_line', 'crescent_gap', 'water_spot', 
                                         'oil_spot', 'silk_spot', 'inclusion', 
                                         'rolled_pit', 'crease'), show='headings',
                    yscrollcommand=tree_scroll.set)

tree_scroll.config(command=tree.yview)

# Treeview 열 제목 설정
tree.heading("name", text='name')
tree.heading("waist_folding", text="waist_folding")
tree.heading("punching_hole", text="punching_hole")
tree.heading("welding_line", text="welding_line")
tree.heading("crescent_gap", text="crescent_gap")
tree.heading("water_spot", text="water_spot")
tree.heading("oil_spot", text="oil_spot")
tree.heading("silk_spot", text="silk_spot")
tree.heading("inclusion", text="inclusion")
tree.heading("rolled_pit", text="rolled_pit")
tree.heading("crease", text="crease")

tree.column("name", width=120)
tree.column("waist_folding", width=100)
tree.column("punching_hole", width=100)
tree.column("welding_line", width=100)
tree.column("crescent_gap", width=100)
tree.column("water_spot", width=100)
tree.column("oil_spot", width=100)
tree.column("silk_spot", width=100)
tree.column("inclusion", width=100)
tree.column("rolled_pit", width=100)
tree.column("crease", width=100)

tree.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)

# 파일을 순차적으로 확인하여 예측을 수행하는 함수
def check_for_new_image():
    global file_list, file_index
    if file_list and file_index < len(file_list):
        file_path = os.path.join(selected_folder, file_list[file_index])
        predict_image(file_path)
        file_index += 1

    # 더 이상 파일이 없으면 종료
    if file_index >= len(file_list):
        result_label.config(text="모든 파일을 처리하였습니다.")
        return

    # 20초마다 이 함수를 다시 호출
    window.after(20000, check_for_new_image)  # 20000 밀리초 = 20초

# 폴더 선택 및 이미지 처리 함수
def select_folder():
    global file_list, file_index, selected_folder
    selected_folder = filedialog.askdirectory()
    if selected_folder:
        file_list = [f for f in os.listdir(selected_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]
        file_index = 0
        if file_list:
            check_for_new_image()

# 배열로부터 이미지를 GUI에 표시하는 함수
def show_image_from_array(image_array):
    img = Image.fromarray(image_array)
    img = img.resize((320, 320)) 
    img_tk = ImageTk.PhotoImage(img)
    image_label.config(image=img_tk)
    image_label.image = img_tk

# Keras 모델을 사용한 예측 함수
def predict_image(file_path):
    try:
        image = cv2.imread(file_path)
        if image is None:
            result_label.config(text=f"이미지를 불러오지 못했습니다: {file_path}")
            return
        
        img_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        img_resized = cv2.resize(img_rgb, (224, 224))  
        img_resized = np.expand_dims(img_resized, axis=0)
        img_resized = img_resized / 127.5 - 1  

        result_data = {defect: 0 for defect in defect_classes}

        for defect, model in models.items():
            prediction = model.predict(img_resized)
            result_data[defect] = prediction[0][0]  

        new_data = (
            os.path.basename(file_path),
            result_data['waist folding'],
            result_data['punching_hole'],
            result_data['welding_line'],
            result_data['crescent_gap'],
            result_data['water_spot'],
            result_data['oil_spot'],
            result_data['silk_spot'],
            result_data['inclusion'],
            result_data['rolled_pit'],
            result_data['crease']
        )

        tree.insert("", tk.END, values=new_data)  

        show_image_from_array(img_rgb)

        # *** 예측된 이미지 저장 ***
        save_image_path = os.path.join(save_path, f"output_{os.path.basename(file_path)}")
        cv2.imwrite(save_image_path, cv2.cvtColor(image, cv2.COLOR_RGB2BGR))  # BGR로 변환하여 저장
        print(f"이미지 저장됨: {save_image_path}")

    except Exception as e:
        result_label.config(text=f"예측 중 오류 발생: {str(e)}")

# 폴더 선택 버튼
folder_button = tk.Button(window, text="폴더 선택 및 분석", command=select_folder)
folder_button.grid(row=4, column=0, pady=10)

# 총합을 보여줄 Treeview (하단에 위치)
summary_tree_frame = tk.Frame(window, height=16)  
summary_tree_frame.grid(row=3, column=0, columnspan=2, padx=10, pady=10, sticky="nsew")

summary_tree = ttk.Treeview(summary_tree_frame, columns=('name', 'waist_folding', 'punching_hole', 
                                             'welding_line', 'crescent_gap', 'water_spot', 
                                             'oil_spot', 'silk_spot', 'inclusion', 
                                             'rolled_pit', 'crease'), show='headings', height=2)

summary_tree.heading("name", text='Total')
summary_tree.heading("waist_folding", text="waist_folding")
summary_tree.heading("punching_hole", text="punching_hole")
summary_tree.heading("welding_line", text="welding_line")
summary_tree.heading("crescent_gap", text="crescent_gap")
summary_tree.heading("water_spot", text="water_spot")
summary_tree.heading("oil_spot", text="oil_spot")
summary_tree.heading("silk_spot", text="silk_spot")
summary_tree.heading("inclusion", text="inclusion")
summary_tree.heading("rolled_pit", text="rolled_pit")
summary_tree.heading("crease", text="crease")

summary_tree.column("name", width=120)
summary_tree.column("waist_folding", width=100)
summary_tree.column("punching_hole", width=100)
summary_tree.column("welding_line", width=100)
summary_tree.column("crescent_gap", width=100)
summary_tree.column("water_spot", width=100)
summary_tree.column("oil_spot", width=100)
summary_tree.column("silk_spot", width=100)
summary_tree.column("inclusion", width=100)
summary_tree.column("rolled_pit", width=100)
summary_tree.column("crease", width=100)

summary_tree.pack(fill=tk.BOTH, expand=True)

window.mainloop()


모델 로드 성공: crease
모델 로드 성공: crescent_gap
모델 로드 성공: oil_spot
모델 로드 성공: rolled_pit
모델 로드 성공: punching_hole
모델 로드 성공: silk_spot
모델 로드 성공: inclusion
모델 로드 성공: waist folding
모델 로드 성공: water_spot
모델 로드 성공: welding_line
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
이미지 저장됨: C:/KDT_SF6/vision_ai/모델/개별모델\output_img_01_424799300_01133.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 923ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 880ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 892ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 913ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 877ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 885ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 885ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 875ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 911ms/step
1/1 ━